In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [6]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from IPython.display import Image
import IPython

from typing import TypedDict, Annotated, Dict, Any
import operator
import json

ModuleNotFoundError: No module named 'langchain_experimental'

In [7]:
class AgentState(TypedDict):
    restaurant: str
    output: str

In [ ]:
"""
This will be updated to a web-crawler to scrape menu's from a given restaurant
"""
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
import requests

search_url = "https://s.jina.ai/"
read_url = "https://r.jina.ai/"

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class ReadInput(BaseModel):
    url: str = Field(description="should be a website URL")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search_tool(query: str) -> str:
    """Search a query online and output the markdown, this is an expensive operation use only when needed"""
    return requests.get(search_url + query)

@tool("read-tool", args_schema=ReadInput, return_direct=True)
def read_tool(url: str) -> str:
    """Return markdown of given URL, less computationally expensive than the search tool"""
    return requests.get(read_url + url)

In [ ]:
tools = [search_tool, read_tool]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_node("output-parser", self.parse_output_to_string)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: "output-parser"}
        )
        graph.add_edge("action", "llm")
        graph.add_edge("output-parser", END)
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def parse_output_to_string(self, state: AgentState):
        message = state['messages'][-1]
        json_output = JsonOutputParser(pydantic_object=Restaurants).invoke(message)
        return {'json_output': json_output}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""You are an AI tasked with classifying compatible diet types from a menu.

----------
{ menu }
----------
You output MUST be structured list of one or more of the provided diet types,
{ diet_types }
----------
DO NOT respond as an AI assistant, only return a list of compatible diet types. You may return no diet types if no fit, or one

EXAMPLE:
["diet_type_1", "diet_type_2", ...]
"""
)

prompt.format(context=context)

In [ ]:
llm = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json"
    )
)

model = Model(llm, tools, system=prompt)

In [ ]:
Image(abot.graph.get_graph().draw_png())

In [ ]:
input = [HumanMessage(content="CRISP Rochester")]
result = abot.graph.invoke({"restaurant": input})